In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms  # 核心数据集和预处理模块
from torch.utils.data import DataLoader  # 数据加载器
import torch.nn.functional as F
import torch.optim as optim


In [6]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(
    root='./data',         # 数据存储路径
    train=True,            # 训练集标记
    download=True,         # 如果本地不存在则下载
    transform=transform    # 应用预处理
)

# 测试集
test_dataset = datasets.MNIST(
    root='./data',
    train=False,           # 测试集标记
    download=True,
    transform=transform
)

batch_size=64

train_loader=DataLoader(batch_size=batch_size,shuffle=True,dataset=train_dataset)
test_loader=DataLoader(batch_size=batch_size,dataset=test_dataset)

print(test_dataset[1000])

(tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -

In [22]:
class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.conv2=nn.Conv2d(32,64,3,1,1)
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.fc=nn.Linear(64*7*7,128)
        self.out=nn.Linear(128,10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,64*7*7)
        x=F.relu(self.fc(x))
        x=self.out(x)
        return x
    
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output,target)
        loss.backward()

        optimizer.step()
        if (batch_idx%100)==0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

def test(model,device,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data,target=data.to(device),target.to(device)
            output=model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # 计算损失
            pred = output.argmax(dim=1, keepdim=True)  # 获取最大概率的索引
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

    


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型
CNNmodel = CNNmodel().to(device)

# 定义优化器
optimizer = optim.Adam(CNNmodel.parameters(), lr=0.001)

# 训练和测试模型
for epoch in range(1, 11):
    train(CNNmodel, device, train_loader, optimizer, epoch)
    test(CNNmodel,device,test_loader)

Train Epoch: 1 [0/60000] Loss: 2.310168
Train Epoch: 1 [6400/60000] Loss: 0.161464
Train Epoch: 1 [12800/60000] Loss: 0.188618
Train Epoch: 1 [19200/60000] Loss: 0.206894
Train Epoch: 1 [25600/60000] Loss: 0.040116
Train Epoch: 1 [32000/60000] Loss: 0.010934
Train Epoch: 1 [38400/60000] Loss: 0.048808
Train Epoch: 1 [44800/60000] Loss: 0.491910
Train Epoch: 1 [51200/60000] Loss: 0.131577
Train Epoch: 1 [57600/60000] Loss: 0.095757

Test set: Average loss: 0.0511, Accuracy: 9830/10000 (98.30%)

Train Epoch: 2 [0/60000] Loss: 0.008132
Train Epoch: 2 [6400/60000] Loss: 0.069837
Train Epoch: 2 [12800/60000] Loss: 0.006585
Train Epoch: 2 [19200/60000] Loss: 0.072716
Train Epoch: 2 [25600/60000] Loss: 0.038847
Train Epoch: 2 [32000/60000] Loss: 0.157350
Train Epoch: 2 [38400/60000] Loss: 0.018967
Train Epoch: 2 [44800/60000] Loss: 0.107969
Train Epoch: 2 [51200/60000] Loss: 0.003945
Train Epoch: 2 [57600/60000] Loss: 0.024176

Test set: Average loss: 0.0333, Accuracy: 9889/10000 (98.89%)

Tr

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, int)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!tuple of (Tensor, int)!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [1]:
(data,target)=test_dataset[999]
print(CNNmodel(data))

NameError: name 'test_dataset' is not defined